In [8]:
def downsample_data(
    lower_df: pd.DataFrame,
    upper_series: Union[pd.DataFrame, pd.Series],
    lower_timeframe: str,
    upper_timeframe: str,
) -> pd.DataFrame:
    """
    Downscales a DataFrame to align with the given upper timeframe DataFrame or Series.

    Args:
        lower_df (pd.DataFrame): The lower timeframe DataFrame.
        upper_series (pd.DataFrame or pd.Series): The upper timeframe DataFrame or Series.
        lower_timeframe (str): The lower timeframe of the input DataFrame.
        upper_timeframe (str): The upper timeframe for alignment.

    Returns:
        pd.DataFrame: A DataFrame where the values from upper_series are aligned with the lower_df.

    Raises:
        ValueError: If the upper_series is empty.
        ValueError: If the frequencies of the timeframes are not compatible.
        ValueError: If the indices are not DatetimeIndex.
    """
    # Validate inputs
    if not isinstance(lower_df.index, pd.DatetimeIndex):
        raise ValueError("lower_df must be indexed by DatetimeIndex")
    if not isinstance(upper_series.index, pd.DatetimeIndex):
        raise ValueError("upper_series must be indexed by DatetimeIndex")

    if upper_series.empty:
        raise ValueError("upper_series cannot be empty")

    # Convert upper_series to DataFrame if it is a Series
    if isinstance(upper_series, pd.Series):
        upper_series = upper_series.to_frame()

    # Check if the lower timeframe is higher frequency than the upper timeframe
    lower_freq = pd.Timedelta(lower_timeframe)
    upper_freq = pd.Timedelta(upper_timeframe)

    if lower_freq >= upper_freq:
        raise ValueError(
            "The lower timeframe must be a higher frequency than the upper timeframe"
        )

    # Resample upper_series to align with lower_df
    resampled_upper_series = upper_series.resample(lower_timeframe).ffill()

    # Shift the resampled upper_series to avoid look-ahead bias
    resampled_upper_series = resampled_upper_series.shift(1)

    # Align the resampled upper_series with the lower_df
    aligned_upper_series = resampled_upper_series.reindex(
        lower_df.index, method="ffill"
    )

    # Concatenate the aligned upper_series with the lower_df
    result_df = pd.concat([lower_df, aligned_upper_series], axis=1)

    return result_df

Time taken to access elements in a 2D list: 0.173210 seconds
Time taken to access elements in a 2D NumPy array: 0.683446 seconds
